In [4]:
import os
import pyspark.sql.functions as F
import pyspark.sql.window as Window
from pyspark.sql import SparkSession

In [5]:
os.environ['SPARK_HOME'] = r'C:\spark\spark-3.5.4-bin-hadoop3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [6]:
spark = (
    SparkSession.builder
    .appName('Airport Traffic')
    .master('local[20]')
    .config('spark.executor.memory', '12g')
    .config('spark.executor.cors', '4')
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.minExecutors", "2")
    .config("spark.dynamicAllocation.maxExecutors", "20")
    .config('spark.executors.memoryOverhead', '2g')
    .config("spark.driver.memory", "12g")
    .config("spark.driver.maxResultSize", "4g")
    .config('spark.sql.adaptive.enabled', 'true')
    .config('spark.sql.adaptive.coalescePartitions.enabled', 'true')
    .config('spark.sql.adaptive.advisoryPartitionSizeInBytes', '128mb')# 128 default
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config('spark.dynamicAllocation.executorIdleTimeout', '120')
    .config('spark.sql.autoBroadcastJoinThreshold', '512mb')
)